In [17]:
import tensorflow as tf
import numpy as np
import random, os, glob, time
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [3]:
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Data and image parameters
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
TRAIN_DIR = 'data/chest_xray/train'
TEST_DIR  = 'data/chest_xray/test'
# Assume training folder has one subfolder per class (e.g., NORMAL, PNEUMONIA)
NUM_CLASSES = len([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
# For episodic (few–shot) training:
K_SHOT  = 5  # number of support images per class per episode
Q_QUERY = 5  # number of query images per class per episode

print(f"Found {NUM_CLASSES} classes in training data.")

Found 2 classes in training data.


In [9]:
def load_data_by_class(data_dir):
    """
    Loads file paths for each class into a dictionary.
    """
    data = {}
    for cls in os.listdir(data_dir):
        full_dir = os.path.join(data_dir, cls)
        if os.path.isdir(full_dir):
            files = glob.glob(os.path.join(full_dir, '*'))
            data[cls] = files
    return data

# Load training and test data by class (for episodic training)
train_data = load_data_by_class(TRAIN_DIR)
test_data  = load_data_by_class(TEST_DIR)

def generate_episode(data, num_classes=NUM_CLASSES, k_shot=K_SHOT, q_query=Q_QUERY):
    """
    Generate one episode:
      - Randomly sample 'num_classes' from available classes.
      - For each, randomly sample k_shot support images and q_query query images.
    Returns:
      support_images, support_labels, query_images, query_labels.
    Labels are one–hot encoded (order is per–episode).
    """
    support_images, support_labels = [], []
    query_images, query_labels = [], []
    selected_classes = random.sample(list(data.keys()), num_classes)
    for i, cls in enumerate(selected_classes):
        images = random.sample(data[cls], k_shot + q_query)
        support_files = images[:k_shot]
        query_files   = images[k_shot:]
        for img_path in support_files:
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img = tf.keras.preprocessing.image.img_to_array(img)
            support_images.append(img)
            label = np.zeros(num_classes)
            label[i] = 1
            support_labels.append(label)
        for img_path in query_files:
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img = tf.keras.preprocessing.image.img_to_array(img)
            query_images.append(img)
            label = np.zeros(num_classes)
            label[i] = 1
            query_labels.append(label)
    # Normalize images:
    support_images = np.array(support_images) / 255.0
    query_images   = np.array(query_images)   / 255.0
    return (support_images, 
        np.array(support_labels, dtype=np.float32), 
        query_images, 
        np.array(query_labels, dtype=np.float32))

In [5]:
def build_original_model():
    base_model = VGG16(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    output = Dense(NUM_CLASSES, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=output)

original_model = build_original_model()
original_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
original_model.summary()

# Train using ImageDataGenerator (note: for demo, epochs are kept low)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen  = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=4, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
    TEST_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=4, class_mode='categorical')

print("\nTraining Original Model...")
t0 = time.time()
original_history = original_model.fit(train_generator,
                                      validation_data=test_generator,
                                      epochs=5,
                                      steps_per_epoch=len(train_generator),
                                      validation_steps=len(test_generator))
original_training_time = time.time() - t0

# Evaluate original model on the test set:
original_eval = original_model.evaluate(test_generator, steps=len(test_generator))
print("\nOriginal Model Evaluation:", original_eval)
y_pred = np.argmax(original_model.predict(test_generator, steps=len(test_generator)), axis=1)
y_true = test_generator.classes
cm_original = confusion_matrix(y_true, y_pred)
report_original = classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys()))
print("Confusion Matrix (Original):\n", cm_original)
print("Classification Report (Original):\n", report_original)

# Save the original model:
original_model.save('models/original_model.h5')
print("Saved original_model.h5")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.

Training Original Model...
Epoch 1/5


/Users/bobbytomlinson/Desktop/Coding/Personal/001/001venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1304/1304 ━━━━━━━━━━━━━━━━━━━━ 380s 291ms/step - accuracy: 0.8945 - loss: 0.3500 - val_accuracy: 0.8462 - val_loss: 0.7920
Epoch 2/5
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 380s 291ms/step - accuracy: 0.9380 - loss: 0.2886 - val_accuracy: 0.9215 - val_loss: 0.5385
Epoch 3/5
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 380s 291ms/step - accuracy: 0.9593 - loss: 0.1939 - val_accuracy: 0.9103 - val_loss: 0.6569
Epoch 4/5
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 375s 288ms/step - accuracy: 0.9610 - loss: 0.2268 - val_accuracy: 0.8942 - val_loss: 1.0403
Epoch 5/5
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 382s 293ms/step - accuracy: 0.9632 - loss: 0.2121 - val_accuracy: 0.8894 - val_loss: 1.3619
156/156 ━━━━━━━━━━━━━━━━━━━━ 43s 277ms/step - accuracy: 0.8910 - loss: 1.4066

Original Model Evaluation: [1.3618521690368652, 0.8894230723381042]
156/156 ━━━━━━━━━━━━━━━━━━━━ 43s 273ms/step
Confusion Matrix (Original):
 [[ 61 173]
 [108 282]]
Classification Report (Original):
               precision    recall  f1-score   support

      NORMA

In [23]:
def build_backbone():
    """
    Build a VGG16–based backbone that outputs an embedding.
    (Note: base layers are frozen.)
    """
    vgg = VGG16(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    embedding = Dense(128, activation='relu', name='embedding')(x)
    return Model(inputs=vgg.input, outputs=embedding, name='Backbone')

backbone_fsl = build_backbone()
backbone_fsl.summary()

Model: "Backbone"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Dense)               │ (None, 128)            │     3,211,392 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,080 (68.38 MB)

 Trainable params: 3,211,392 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [24]:
def get_fresh_backbone():
    return build_backbone()

In [25]:
# -----------------------------------------------------------------------------
# 5a. MAML (Model–Agnostic Meta–Learning)
# -----------------------------------------------------------------------------
def build_maml_model(backbone, num_classes=NUM_CLASSES):
    """
    Build a complete classifier: backbone + classifier head.
    """
    inp = backbone.input
    x = backbone.output
    out = Dense(num_classes, activation='softmax', name='classifier')(x)
    return Model(inputs=inp, outputs=out, name='MAML_Model')

maml_backbone = get_fresh_backbone()
maml_model = build_maml_model(maml_backbone)
maml_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# In graph mode we use tensor–based copies of the weights.
@tf.function
def maml_train_step(model, support_imgs, support_lbls, query_imgs, query_lbls, inner_lr=0.01):
    # Save original weights as tensor copies.
    original_weights = [tf.identity(w) for w in model.trainable_variables]
    
    # --- Inner Loop: Adaptation on support set ---
    with tf.GradientTape() as tape:
        support_preds = model(support_imgs, training=True)
        support_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(support_lbls, support_preds))
    grads = tape.gradient(support_loss, model.trainable_variables)
    for var, grad in zip(model.trainable_variables, grads):
        var.assign_sub(inner_lr * grad)
    
    # --- Outer Loop: Meta–loss on query set ---
    with tf.GradientTape() as tape:
        query_preds = model(query_imgs, training=True)
        query_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(query_lbls, query_preds))
    meta_grads = tape.gradient(query_loss, model.trainable_variables)
    maml_optimizer.apply_gradients(zip(meta_grads, model.trainable_variables))
    
    # Restore original weights (so each episode is independent)
    for var, orig in zip(model.trainable_variables, original_weights):
        var.assign(orig)
    
    return support_loss, query_loss

# Train MAML episodically (using a reduced number of episodes for demo)

maml_episodes = 50
print("\nTraining MAML...")
t0 = time.time()
for episode in range(maml_episodes):
    s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(train_data)
    s_loss, q_loss = maml_train_step(maml_model, s_imgs, s_lbls, q_imgs, q_lbls)
    if episode % 10 == 0:
        print(f"MAML Episode {episode}: Support Loss = {s_loss.numpy():.4f}, Query Loss = {q_loss.numpy():.4f}")
maml_training_time = time.time() - t0

def evaluate_maml(model, data, num_episodes=20):
    total_correct = 0
    total_samples = 0
    for _ in range(num_episodes):
        s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(data)
        # For evaluation, simulate one inner-loop update:
        adapted_model = tf.keras.models.clone_model(model)
        adapted_model.set_weights(model.get_weights())
        with tf.GradientTape() as tape:
            s_preds = adapted_model(s_imgs, training=False)
            s_loss  = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(s_lbls, s_preds))
        grads = tape.gradient(s_loss, adapted_model.trainable_variables)
        for var, grad in zip(adapted_model.trainable_variables, grads):
            var.assign_sub(0.01 * grad)
        q_preds = adapted_model(q_imgs, training=False)
        pred_classes = tf.argmax(q_preds, axis=1)
        true_classes = tf.argmax(q_lbls, axis=1)
        total_correct += np.sum(pred_classes.numpy() == true_classes.numpy())
        total_samples += q_imgs.shape[0]
    return total_correct / total_samples

maml_accuracy = evaluate_maml(maml_model, test_data)
print("MAML Evaluation Accuracy:", maml_accuracy)

maml_model.save('models/maml_model.h5')
print("Saved maml_model.h5")


Training MAML...
MAML Episode 0: Support Loss = 0.6472, Query Loss = 2.8090
MAML Episode 10: Support Loss = 0.7144, Query Loss = 2.3006
MAML Episode 20: Support Loss = 0.6684, Query Loss = 0.8695
MAML Episode 30: Support Loss = 0.6718, Query Loss = 0.8222
MAML Episode 40: Support Loss = 0.8074, Query Loss = 2.8041


MAML Evaluation Accuracy: 0.505
Saved maml_model.h5


In [26]:
# -----------------------------------------------------------------------------
# 5b. Matching Networks
# -----------------------------------------------------------------------------

matching_backbone = get_fresh_backbone()
def matching_network_prediction(embedding_model, support_imgs, support_lbls, query_imgs):
    """
    Compute cosine similarity between query and support embeddings,
    then return the weighted sum of support labels as prediction.
    """
    support_embeddings = embedding_model(support_imgs)  # shape: (n_support, emb_dim)
    query_embeddings   = embedding_model(query_imgs)    # shape: (n_query, emb_dim)
    support_norm = tf.nn.l2_normalize(support_embeddings, axis=1)
    query_norm   = tf.nn.l2_normalize(query_embeddings, axis=1)
    sims = tf.matmul(query_norm, support_norm, transpose_b=True)  # (n_query, n_support)
    attention = tf.nn.softmax(sims, axis=1)
    predictions = tf.matmul(attention, support_lbls)
    return predictions

matching_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def matching_train_step(embedding_model, support_imgs, support_lbls, query_imgs, query_lbls):
    with tf.GradientTape() as tape:
        preds = matching_network_prediction(embedding_model, support_imgs, support_lbls, query_imgs)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(query_lbls, preds))
    grads = tape.gradient(loss, embedding_model.trainable_variables)
    matching_optimizer.apply_gradients(zip(grads, embedding_model.trainable_variables))
    return loss

# Train Matching Networks episodically
matching_episodes = 50
print("\nTraining Matching Networks...")
t0 = time.time()
for episode in range(matching_episodes):
    s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(train_data)
    loss = matching_train_step(matching_backbone, s_imgs, s_lbls, q_imgs, q_lbls)
    if episode % 10 == 0:
        print(f"Matching Networks Episode {episode}: Loss = {loss.numpy():.4f}")
matching_training_time = time.time() - t0

def evaluate_matching(embedding_model, data, num_episodes=20):
    total_correct = 0
    total_samples = 0
    for _ in range(num_episodes):
        s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(data)
        preds = matching_network_prediction(embedding_model, s_imgs, s_lbls, q_imgs)
        pred_classes = tf.argmax(preds, axis=1)
        true_classes = tf.argmax(q_lbls, axis=1)
        total_correct += np.sum(pred_classes.numpy() == true_classes.numpy())
        total_samples += q_imgs.shape[0]
    return total_correct / total_samples

matching_accuracy = evaluate_matching(matching_backbone, test_data)
print("Matching Networks Evaluation Accuracy:", matching_accuracy)

matching_backbone.save('models/matching_backbone.h5')
print("Saved matching_backbone.h5")


Training Matching Networks...
Matching Networks Episode 0: Loss = 0.6870
Matching Networks Episode 10: Loss = 0.3703
Matching Networks Episode 20: Loss = 0.4227
Matching Networks Episode 30: Loss = 0.3554
Matching Networks Episode 40: Loss = 0.5474


Matching Networks Evaluation Accuracy: 0.655
Saved matching_backbone.h5


In [27]:
# -----------------------------------------------------------------------------
# 5c. Prototypical Networks
# -----------------------------------------------------------------------------

proto_backbone = get_fresh_backbone()


def prototypical_network_prediction(embedding_model, support_imgs, support_lbls, query_imgs, num_classes=NUM_CLASSES):
    support_embeddings = embedding_model(support_imgs)
    query_embeddings   = embedding_model(query_imgs)
    prototypes = []
    for c in range(num_classes):
        class_mask = tf.equal(tf.argmax(support_lbls, axis=1), c)
        class_embeddings = tf.boolean_mask(support_embeddings, class_mask)
        proto = tf.reduce_mean(class_embeddings, axis=0)
        prototypes.append(proto)
    prototypes = tf.stack(prototypes)  # shape: (num_classes, emb_dim)
    distances = tf.reduce_sum(tf.square(tf.expand_dims(query_embeddings, axis=1) - prototypes), axis=2)
    logits = -distances  # negative Euclidean distance
    predictions = tf.nn.softmax(logits, axis=1)
    return predictions

proto_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def proto_train_step(embedding_model, support_imgs, support_lbls, query_imgs, query_lbls):
    with tf.GradientTape() as tape:
        preds = prototypical_network_prediction(embedding_model, support_imgs, support_lbls, query_imgs)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(query_lbls, preds))
    grads = tape.gradient(loss, embedding_model.trainable_variables)
    proto_optimizer.apply_gradients(zip(grads, embedding_model.trainable_variables))
    return loss

# Train Prototypical Networks episodically
proto_episodes = 50
print("\nTraining Prototypical Networks...")
t0 = time.time()
for episode in range(proto_episodes):
    s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(train_data)
    loss = proto_train_step(proto_backbone, s_imgs, s_lbls, q_imgs, q_lbls)
    if episode % 10 == 0:
        print(f"Prototypical Networks Episode {episode}: Loss = {loss.numpy():.4f}")
proto_training_time = time.time() - t0

def evaluate_proto(embedding_model, data, num_episodes=20):
    total_correct = 0
    total_samples = 0
    for _ in range(num_episodes):
        s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(data)
        preds = prototypical_network_prediction(embedding_model, s_imgs, s_lbls, q_imgs)
        pred_classes = tf.argmax(preds, axis=1)
        true_classes = tf.argmax(q_lbls, axis=1)
        total_correct += np.sum(pred_classes.numpy() == true_classes.numpy())
        total_samples += q_imgs.shape[0]
    return total_correct / total_samples

proto_accuracy = evaluate_proto(proto_backbone, test_data)
print("Prototypical Networks Evaluation Accuracy:", proto_accuracy)

proto_backbone.save('models/proto_backbone.h5')
print("Saved proto_backbone.h5")


Training Prototypical Networks...
Prototypical Networks Episode 0: Loss = 0.4226
Prototypical Networks Episode 10: Loss = 7.4394
Prototypical Networks Episode 20: Loss = 2.8208
Prototypical Networks Episode 30: Loss = 0.0000
Prototypical Networks Episode 40: Loss = 2.4652


Prototypical Networks Evaluation Accuracy: 0.885
Saved proto_backbone.h5


In [28]:
relation_backbone = get_fresh_backbone()


# -----------------------------------------------------------------------------
# 5d. Relation Networks
# -----------------------------------------------------------------------------
def build_relation_module(input_dim):
    inp = Input(shape=(input_dim,))
    x = Dense(64, activation='relu')(inp)
    out = Dense(1, activation='sigmoid')(x)
    return Model(inputs=inp, outputs=out, name='Relation_Module')

# Relation module takes concatenated [query_embedding, prototype] (i.e. 2*embedding_dim)
relation_module = build_relation_module(128*2)
relation_module.summary()
relation_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def relation_train_step(embedding_model, relation_module, support_imgs, support_lbls, query_imgs, query_lbls, num_classes=NUM_CLASSES):
    with tf.GradientTape() as tape:
        support_embeddings = embedding_model(support_imgs)
        query_embeddings   = embedding_model(query_imgs)
        prototypes = []
        for c in range(num_classes):
            class_mask = tf.equal(tf.argmax(support_lbls, axis=1), c)
            class_embeddings = tf.boolean_mask(support_embeddings, class_mask)
            proto = tf.reduce_mean(class_embeddings, axis=0, keepdims=True)
            prototypes.append(proto)
        prototypes = tf.concat(prototypes, axis=0)  # shape: (num_classes, emb_dim)
        relation_scores = []
        # For each class, compute a relation score between the query embeddings and the prototype.
        for c in range(num_classes):
            proto = tf.expand_dims(prototypes[c], axis=0)  # (1, emb_dim)
            # Tile the prototype to the query batch size:
            proto_tiled = tf.tile(proto, [tf.shape(query_embeddings)[0], 1])
            pair = tf.concat([query_embeddings, proto_tiled], axis=1)
            score = relation_module(pair)  # (n_query, 1)
            relation_scores.append(score)
        relation_scores = tf.concat(relation_scores, axis=1)  # (n_query, num_classes)
        predictions = tf.nn.softmax(relation_scores, axis=1)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(query_lbls, predictions))
    vars_to_update = embedding_model.trainable_variables + relation_module.trainable_variables
    grads = tape.gradient(loss, vars_to_update)
    relation_optimizer.apply_gradients(zip(grads, vars_to_update))
    return loss

# Train Relation Networks episodically
relation_episodes = 50
print("\nTraining Relation Networks...")
t0 = time.time()
for episode in range(relation_episodes):
    s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(train_data)
    loss = relation_train_step(relation_backbone, relation_module, s_imgs, s_lbls, q_imgs, q_lbls)
    if episode % 10 == 0:
        print(f"Relation Networks Episode {episode}: Loss = {loss.numpy():.4f}")
relation_training_time = time.time() - t0

def evaluate_relation(embedding_model, relation_module, data, num_episodes=20, num_classes=NUM_CLASSES):
    total_correct = 0
    total_samples = 0
    for _ in range(num_episodes):
        s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(data)
        support_embeddings = embedding_model(s_imgs)
        query_embeddings   = embedding_model(q_imgs)
        prototypes = []
        for c in range(num_classes):
            class_mask = tf.equal(tf.argmax(s_lbls, axis=1), c)
            class_embeddings = tf.boolean_mask(support_embeddings, class_mask)
            proto = tf.reduce_mean(class_embeddings, axis=0, keepdims=True)
            prototypes.append(proto)
        prototypes = tf.concat(prototypes, axis=0)
        relation_scores = []
        for c in range(num_classes):
            proto = tf.expand_dims(prototypes[c], axis=0)
            proto_tiled = tf.tile(proto, [tf.shape(query_embeddings)[0], 1])
            pair = tf.concat([query_embeddings, proto_tiled], axis=1)
            score = relation_module(pair)
            relation_scores.append(score)
        relation_scores = tf.concat(relation_scores, axis=1)
        predictions = tf.nn.softmax(relation_scores, axis=1)
        pred_classes = tf.argmax(predictions, axis=1)
        true_classes = tf.argmax(q_lbls, axis=1)
        total_correct += np.sum(pred_classes.numpy() == true_classes.numpy())
        total_samples += q_imgs.shape[0]
    return total_correct / total_samples

relation_accuracy = evaluate_relation(relation_backbone, relation_module, test_data)
print("Relation Networks Evaluation Accuracy:", relation_accuracy)

relation_backbone.save('models/relation_backbone.h5')
relation_module.save('models/relation_module.h5')
print("Saved relation_backbone.h5 and relation_module.h5")

Model: "Relation_Module"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,513 (64.50 KB)

 Trainable params: 16,513 (64.50 KB)

 Non-trainable params: 0 (0.00 B)


Training Relation Networks...
Relation Networks Episode 0: Loss = 0.6933
Relation Networks Episode 10: Loss = 0.6859
Relation Networks Episode 20: Loss = 0.5735
Relation Networks Episode 30: Loss = 0.5774
Relation Networks Episode 40: Loss = 0.6354


Relation Networks Evaluation Accuracy: 0.59
Saved relation_backbone.h5 and relation_module.h5


In [29]:
# -----------------------------------------------------------------------------
# 6. Comparative Results Summary
# -----------------------------------------------------------------------------
results = {
    "Original Model": {
        "Accuracy": original_eval[1],
        "Training Time (s)": original_training_time,
        "Confusion Matrix": cm_original,
        "Classification Report": report_original,
    },
    "MAML": {
        "Accuracy": maml_accuracy,
        "Training Time (s)": maml_training_time,
    },
    "Matching Networks": {
        "Accuracy": matching_accuracy,
        "Training Time (s)": matching_training_time,
    },
    "Prototypical Networks": {
        "Accuracy": proto_accuracy,
        "Training Time (s)": proto_training_time,
    },
    "Relation Networks": {
        "Accuracy": relation_accuracy,
        "Training Time (s)": relation_training_time,
    }
}

print("\n=== Comparative Results ===")
for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for key, value in metrics.items():
        print(f"  {key}: {value}")


=== Comparative Results ===

Method: Original Model
  Accuracy: 0.8894230723381042
  Training Time (s): 1896.089072227478
  Confusion Matrix: [[ 61 173]
 [108 282]]
  Classification Report:               precision    recall  f1-score   support

      NORMAL       0.36      0.26      0.30       234
   PNEUMONIA       0.62      0.72      0.67       390

    accuracy                           0.55       624
   macro avg       0.49      0.49      0.49       624
weighted avg       0.52      0.55      0.53       624


Method: MAML
  Accuracy: 0.505
  Training Time (s): 67.7479157447815

Method: Matching Networks
  Accuracy: 0.655
  Training Time (s): 70.79149127006531

Method: Prototypical Networks
  Accuracy: 0.885
  Training Time (s): 67.75871014595032

Method: Relation Networks
  Accuracy: 0.59
  Training Time (s): 68.0609540939331


In [30]:
results = {
    "Original Model": {
        "Accuracy": original_eval[1],
        "Training Time (s)": original_training_time,
        "Confusion Matrix": str(cm_original),
        "Classification Report": report_original,
    },
    "MAML": {
        "Accuracy": maml_accuracy,
        "Training Time (s)": maml_training_time,
    },
    "Matching Networks": {
        "Accuracy": matching_accuracy,
        "Training Time (s)": matching_training_time,
    },
    "Prototypical Networks": {
        "Accuracy": proto_accuracy,
        "Training Time (s)": proto_training_time,
    },
    "Relation Networks": {
        "Accuracy": relation_accuracy,
        "Training Time (s)": relation_training_time,
    }
}

# Convert dictionary to DataFrame and export to CSV:
rows = []
for method, metrics in results.items():
    row = {"Method": method}
    for key, value in metrics.items():
        row[key] = value
    rows.append(row)

df = pd.DataFrame(rows)
csv_filename = "fsl_comparison_metrics.csv"
df.to_csv(csv_filename, index=False)
print(f"\nExported metrics to {csv_filename}")


Exported metrics to fsl_comparison_metrics.csv
